# <a id='toc1_'></a>[Modèle sur mesure avancé](#toc0_)


**Table of contents**<a id='toc0_'></a>    
- [Modèle sur mesure avancé](#toc1_)    
- [Telechargements & imports des données](#toc2_)    
- [Preprocessing des données](#toc3_)    
  - [Renommer les colonnes](#toc3_1_)    
  - [Text cleaning](#toc3_2_)    
  - [Fonctions de split](#toc3_3_)    
    - [Lemmatisation](#toc3_3_1_)    
    - [Stemmatisation](#toc3_3_2_)    
    - [Tokenisation](#toc3_3_3_)    
  - [Definir Features et Target](#toc3_4_)    
  - [Embedding et vectorisation](#toc3_5_)    
    - [keras embedding](#toc3_5_1_)    
    - [Glove Embedding](#toc3_5_2_)    
- [Modelisation](#toc4_)    
  - [Log des Models avec MlFlow](#toc4_1_)    
  - [Creations des modèles](#toc4_2_)    
    - [RNN](#toc4_2_1_)    
    - [LSTM](#toc4_2_2_)    
    - [LSTM BIDIRECTIONEL](#toc4_2_3_)    
  - [Entrainement et evaluation des modèles](#toc4_3_)    
    - [Tokenisation avec Keras](#toc4_3_1_)    
    - [Tokenisation avec Glove](#toc4_3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc2_'></a>[Telechargements & imports des données](#toc0_)

In [ ]:
!pip install uv
!uv pip install pandas numpy matplotlib scikit-learn wordcloud tqdm sentence_transformers ipykernel tensorflow spacy mlflow
!python -m spacy download en_core_web_sm

In [ ]:
import os
import re
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from wordcloud import WordCloud

tqdm.pandas()

import string
import warnings

warnings.filterwarnings("ignore")
import mlflow
import mlflow.tensorflow
import nltk
import spacy
import tensorflow as tf
from nltk.stem.snowball import PorterStemmer
from sentence_transformers import SentenceTransformer
from sklearn.metrics import (
    average_precision_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from transformers import AutoModelForSequenceClassification, AutoTokenizer

nltk.download('punkt')
nltk.download('punkt_tab')


nlp = spacy.load("en_core_web_sm")
stemmer = PorterStemmer()

import mlflow
import mlflow.sklearn
import mlflow.tensorflow
import pandas as pd
from mlflow.models.signature import infer_signature
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    average_precision_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from tensorflow.keras.layers import (
    LSTM,
    Bidirectional,
    Dense,
    Dropout,
    Embedding,
    SimpleRNN,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
# Telecharger les données
!wget https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip

# Extraction des données
ZIP_PATH = '/content/sentiment140.zip'
!unzip $ZIP_PATH

In [ ]:
# Lecture du Dataframe
DATASET_PATH = '/content/training.1600000.processed.noemoticon.csv'
df = pd.read_csv(DATASET_PATH, sep=',', encoding = "ISO-8859-1", header=None)
df.head()

# <a id='toc3_'></a>[Preprocessing des données](#toc0_)

## <a id='toc3_1_'></a>[Renommer les colonnes](#toc0_)

In [ ]:
# Renommer les colonnes en ce basant sur les cards du dataset
df = df.rename(columns={
    df.columns[0]: 'target',
    df.columns[1]: 'ids',
    df.columns[2]: 'date',
    df.columns[3]: 'flag',
    df.columns[4]: 'user',
    df.columns[5]: 'text',

})

In [ ]:
# Definir les jeux de données

complete_df = df[['target', 'text']]
sample_df = df[['target', 'text']].sample(16_000)

# Afficher la valeurs des labels initiaux
print(sample_df['target'].value_counts())

# Conversion en binaire 0,1
sample_df['target'] = sample_df['target'].replace({0: 0, 4: 1})
complete_df['target'] = complete_df['target'].replace({0: 0, 4: 1})


## <a id='toc3_2_'></a>[Text cleaning](#toc0_)

In [ ]:

def tweet_cleaning(tweet):
    """
    Nettoie et prétraite un tweet

    Cette fonction effectue plusieurs étapes de nettoyage :
        - Suppression des URLs, mentions et hashtags
        - Suppression des emojis et caractères spéciaux
        - Suppression de la ponctuation et des chiffres
        - Normalisation du texte (minuscules, espaces multiples)

    Params :
        tweet (str) : Le tweet brut à nettoyer.

    Return :
        str : Le tweet nettoyé et prétraité, prêt pour l'analyse de sentiment.

    """
    # Supprimer les URLs
    tweet = re.sub(r'https?://\S+|www\.\S+', '', tweet)

    # Supprimer les mentions (@user)
    tweet = re.sub(r'@\w+', '', tweet)

    # Supprimer les hashtags (#hashtag)
    tweet = re.sub(r'#\w+', '', tweet)

    # Normaliser & supprimer les caractères
    tweet = tweet.encode('ascii', 'ignore').decode('utf-8')
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)

    # Supprimer la ponctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Supprimer les chiffres
    tweet = re.sub(r'\d+', '', tweet)

    # Supprimer les espaces multiples et les espaces au début/fin
    tweet = re.sub(r'\s+', ' ', tweet).strip()

    return tweet



In [ ]:
# appliquer la fonction a la colonne text
sample_df.apply(lambda x: tweet_cleaning(x['text']), axis=1)

## <a id='toc3_3_'></a>[Fonctions de split](#toc0_)

### <a id='toc3_3_1_'></a>[Lemmatisation](#toc0_)

In [ ]:
def lemmatize_text(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc]

### <a id='toc3_3_2_'></a>[Stemmatisation](#toc0_)

In [ ]:
def stem_text_french(text):
    tokens = nltk.word_tokenize(text, language='french')
    return [stemmer.stem(token) for token in tokens]

### <a id='toc3_3_3_'></a>[Tokenisation](#toc0_)

In [ ]:
def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc]

In [ ]:
# Exemple de resultat sur 1 sample
text = sample_df['text'].sample(1).values[0]
print(text)

print(lemmatize_text(text=text))
print(stem_text_french(text=text))
print(tokenize_text(text=text))

## <a id='toc3_4_'></a>[Definir Features et Target](#toc0_)

In [ ]:
X = sample_df['text'].apply(tweet_cleaning)
y = sample_df['target']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## <a id='toc3_5_'></a>[Embedding et vectorisation](#toc0_)

### <a id='toc3_5_1_'></a>[keras embedding](#toc0_)

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10_000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding
MAX_LEN = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN)

### <a id='toc3_5_2_'></a>[Glove Embedding](#toc0_)

In [ ]:
!wget -O glove6b100dtxt.zip https://www.kaggle.com/api/v1/datasets/download/danielwillgeorge/glove6b100dtxt
!unzip glove6b100dtxt.zip

In [ ]:
# Charger les embeddings GloVe
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index



In [ ]:
glove_file = 'glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_file)

# Créer une matrice d'embeddings
embedding_matrix = np.zeros((10000, 100))
for word, i in tokenizer.word_index.items():
    if i < 10000:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# <a id='toc4_'></a>[Modelisation](#toc0_)

## <a id='toc4_1_'></a>[Log des Models avec MlFlow](#toc0_)

In [ ]:
def log_model_with_mlflow(model, X_train, y_train, X_test, y_test, tags, model_name, model_version=None, experiment_name="Modèle sur mesure avancé", hyperparams=None):
    """
    Enregistre un modèle TensorFlow avec MLflow, incluant les métriques, les hyperparamètres,
    les tags et le modèle.

    Cette fonction configure une expérience MLflow, démarre une nouvelle exécution, évalue 
    les performances du modèle sur des données de test, enregistre les métriques clés 
    (accuracy, precision, recall, F1, ROC AUC, PR AUC), les hyperparamètres, les tags 
    ainsi que le modèle au format MLflow.

    Args:
        model (tf.keras.Model): Modèle TensorFlow à logger.
        X_train (array-like or pd.DataFrame): Données d'entraînement utilisées pour inférer la signature du modèle.
        y_train (array-like or pd.Series): Labels d'entraînement (utilisés uniquement pour l'évaluation si nécessaire).
        X_test (array-like or pd.DataFrame): Données de test pour l'évaluation du modèle.
        y_test (array-like or pd.Series): Labels de test pour l'évaluation du modèle.
        tags (dict): Dictionnaire de tags supplémentaires à associer à l'exécution MLflow.
        model_name (str): Nom du modèle à utiliser comme nom de l'exécution MLflow.
        model_version (str, optional): Version du modèle à enregistrer comme tag. Par défaut None.
        experiment_name (str, optional): Nom de l'expérience MLflow. Par défaut "Modèle sur mesure avancé".
        hyperparams (dict, optional): Hyperparamètres du modèle à logger. Si None, tente de les extraire automatiquement.
            Si cela échoue, un message est affiché mais l'exécution continue. Par défaut None.

    Returns:
        None: Les résultats sont directement enregistrés dans MLflow.
    """
    
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=model_name):
        # Récupération des hyperparamètres
        if hyperparams is None:
            hyperparams = {}
            try:
                config = model.get_config()
                hyperparams["layers"] = str(config.get("layers", "N/A"))
                hyperparams["optimizer"] = str(model.optimizer.get_config())
            except Exception as e:
                print(f"Impossible de récupérer les hyperparamètres automatiquement : {e}")

        for key, value in hyperparams.items():
            mlflow.log_param(key, value)


        # Prédiction des classes et des probabilités
        y_pred_proba = model.predict(X_test).ravel()
        y_pred_class = (y_pred_proba > 0.5).astype("int32")

        # Calcul des métriques
        accuracy = accuracy_score(y_test, y_pred_class)
        precision = precision_score(y_test, y_pred_class)
        recall = recall_score(y_test, y_pred_class)
        f1 = f1_score(y_test, y_pred_class)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        pr_auc = average_precision_score(y_test, y_pred_proba)

        # Log des métriques dans MLflow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1", f1)
        mlflow.log_metric("roc_auc", roc_auc)
        mlflow.log_metric("pr_auc", pr_auc)

        # Informations sur le modèle (nom et version)
        mlflow.set_tag("mlflow.note.content", model_name)
        if model_version:
            mlflow.set_tag("model_version", model_version)

        # Logger le modèle
        signature = infer_signature(X_train[:2], model.predict(X_train[:2]))
        input_ex = X_train[:2]
        mlflow.tensorflow.log_model(
            model, "model",
            signature=signature,
            input_example=input_ex
        )

        # Ajouter les tags supplémentaires
        for key, val in tags.items():
            mlflow.set_tag(key, val)

In [ ]:
# Pour faire tourner une instance MlFlow sur google Colab

# import os

# # Vérifie si MLflow tourne déjà (simple check basé sur ps)
# status = os.popen("ps aux | grep mlflow").read()
# if "mlflow" not in status:
#     os.system("mlflow ui --port 11247 &")
#     print("MLflow lancé.")
# else:
#     print("MLflow déjà en cours.")

# # Affichage du lien
# from google.colab.output import eval_js

# print("🔗 MLflow UI :", eval_js("google.colab.kernel.proxyPort(11247)"))

## <a id='toc4_2_'></a>[Creations des modèles](#toc0_)

### <a id='toc4_2_1_'></a>[RNN](#toc0_)

In [ ]:
# Modèle RNN avec Keras Embedding
rnn_model_keras = Sequential()
rnn_model_keras.add(Embedding(input_dim=10_000, output_dim=128, input_length=MAX_LEN))
rnn_model_keras.add(SimpleRNN(128))
rnn_model_keras.add(Dense(1, activation='sigmoid'))

rnn_model_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Modèle RNN avec GloVe Embedding
rnn_model_glove = Sequential()
rnn_model_glove.add(Embedding(input_dim=10_000, output_dim=100, input_length=MAX_LEN, weights=[embedding_matrix], trainable=False))
rnn_model_glove.add(SimpleRNN(128))
rnn_model_glove.add(Dense(1, activation='sigmoid'))

rnn_model_glove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### <a id='toc4_2_2_'></a>[LSTM](#toc0_)

In [ ]:
# Modèle LSTM avec Keras Embedding
lstm_model_keras = Sequential()
lstm_model_keras.add(Embedding(input_dim=10_000, output_dim=128, input_length=MAX_LEN))
lstm_model_keras.add(LSTM(128))
lstm_model_keras.add(Dense(1, activation='sigmoid'))

lstm_model_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Modèle LSTM avec GloVe Embedding
lstm_model_glove = Sequential()
lstm_model_glove.add(Embedding(input_dim=10_000, output_dim=100, input_length=MAX_LEN, weights=[embedding_matrix], trainable=False))
lstm_model_glove.add(LSTM(128))
lstm_model_glove.add(Dense(1, activation='sigmoid'))

lstm_model_glove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### <a id='toc4_2_3_'></a>[LSTM BIDIRECTIONEL](#toc0_)

In [ ]:
# Modèle LSTM Bidirectionnel à plusieurs couches avec Keras Embedding
lstm_model_bidirectional_keras = Sequential()
lstm_model_bidirectional_keras.add(Embedding(input_dim=10000, output_dim=128, input_length=MAX_LEN))
lstm_model_bidirectional_keras.add(Bidirectional(LSTM(128, return_sequences=True)))
lstm_model_bidirectional_keras.add(Dropout(0.5))
lstm_model_bidirectional_keras.add(Bidirectional(LSTM(64)))
lstm_model_bidirectional_keras.add(Dropout(0.5))
lstm_model_bidirectional_keras.add(Dense(1, activation='sigmoid'))

lstm_model_bidirectional_keras.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Modèle LSTM Bidirectionnel à plusieurs couches avec GloVe Embedding
lstm_model_bidirectional_glove = Sequential()
lstm_model_bidirectional_glove.add(Embedding(input_dim=10000, output_dim=100, input_length=MAX_LEN, weights=[embedding_matrix], trainable=False))
lstm_model_bidirectional_glove.add(Bidirectional(LSTM(128, return_sequences=True)))
lstm_model_bidirectional_glove.add(Dropout(0.5))
lstm_model_bidirectional_glove.add(Bidirectional(LSTM(64)))
lstm_model_bidirectional_glove.add(Dropout(0.5))
lstm_model_bidirectional_glove.add(Dense(1, activation='sigmoid'))

lstm_model_bidirectional_glove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## <a id='toc4_3_'></a>[Entrainement et evaluation des modèles](#toc0_)

### <a id='toc4_3_1_'></a>[Tokenisation avec Keras](#toc0_)

In [ ]:
# Entraîner les modèles
print('Entrainement du RNN ...')
rnn_model_keras.fit(X_train_pad, y_train, epochs=2, batch_size=256, validation_data=(X_test_pad, y_test))
rnn_loss_keras, rnn_accuracy_keras = rnn_model_keras.evaluate(X_test_pad, y_test)
print(f'RNN Model with Keras Embedding - Loss: {rnn_loss_keras}, Accuracy: {rnn_accuracy_keras}')

print('\n\nEntrainement du LSTM ...')
lstm_model_keras.fit(X_train_pad, y_train, epochs=2, batch_size=256, validation_data=(X_test_pad, y_test))
lstm_loss_keras, lstm_accuracy_keras = lstm_model_keras.evaluate(X_test_pad, y_test)
print(f'LSTM Model with Keras Embedding - Loss: {lstm_loss_keras}, Accuracy: {lstm_accuracy_keras}')

print('\n\nEntrainement du LSTM Bidirectionel ...')
lstm_model_bidirectional_keras.fit(X_train_pad, y_train, epochs=2, batch_size=256, validation_data=(X_test_pad, y_test))
lstm_loss_bidirectional_keras, lstm_accuracy_bidirectional_keras = lstm_model_bidirectional_keras.evaluate(X_test_pad, y_test)
print(f'LSTM Bidirectional Model with Keras Embedding - Loss: {lstm_loss_bidirectional_keras}, Accuracy: {lstm_accuracy_bidirectional_keras}')



In [ ]:
# Logger les modèles avec MLflow

tags = {
    "dataset_used": "sentiment140",
    "embedding_method": "Keras_embedding",
    "preprocessing": "tweet_cleaning_function",
    "sample_size": str(sample_df.shape[0]),
    "sample_seed": "42",
}

# RNN
log_model_with_mlflow(
    model=rnn_model_keras,
    X_train=X_train_pad,
    y_train=y_train,
    X_test=X_test_pad,
    y_test=y_test,
    tags=tags,
    model_name="SimpleRNN_keras_embedding",
    model_version="1",
)

# LSTM
log_model_with_mlflow(
    model=lstm_model_keras,
    X_train=X_train_pad,
    y_train=y_train,
    X_test=X_test_pad,
    y_test=y_test,
    tags=tags,
    model_name="LSTM_keras_embedding",
    model_version="1",
)

# LSTM-BiDir
log_model_with_mlflow(
    model=lstm_model_bidirectional_keras,
    X_train=X_train_pad,
    y_train=y_train,
    X_test=X_test_pad,
    y_test=y_test,
    tags=tags,
    model_name="Bidirectional_LSTM_keras_embedding",
    model_version="1",
)

### <a id='toc4_3_2_'></a>[Tokenisation avec Glove](#toc0_)

In [ ]:
print('Entrainement du RNN ...')
rnn_model_glove.fit(X_train_pad, y_train, epochs=2, batch_size=256, validation_data=(X_test_pad, y_test))
rnn_loss_glove, rnn_accuracy_glove = rnn_model_glove.evaluate(X_test_pad, y_test)
print(f'RNN Model with GloVe Embedding - Loss: {rnn_loss_glove}, Accuracy: {rnn_accuracy_glove}')

print('\n\nEntrainement du LSTM ...')
lstm_model_glove.fit(X_train_pad, y_train, epochs=2, batch_size=256, validation_data=(X_test_pad, y_test))
lstm_loss_glove, lstm_accuracy_glove = lstm_model_glove.evaluate(X_test_pad, y_test)
print(f'LSTM Model with GloVe Embedding - Loss: {lstm_loss_glove}, Accuracy: {lstm_accuracy_glove}')


print('\n\nEntrainement du LSTM Bidirectionel ...')
lstm_model_bidirectional_glove.fit(X_train_pad, y_train, epochs=2, batch_size=256, validation_data=(X_test_pad, y_test))
lstm_loss_bidirectional_glove, lstm_accuracy_bidirectional_glove = lstm_model_bidirectional_glove.evaluate(X_test_pad, y_test)
print(f'LSTM Bidirectional Model with GloVe Embedding - Loss: {lstm_loss_bidirectional_glove}, Accuracy: {lstm_accuracy_bidirectional_glove}')

In [ ]:
# Logger les modèles avec MLflow

tags = {
    "dataset_used": "sentiment140",
    "embedding_method": "Glove_embedding",
    "preprocessing": "tweet_cleaning_function",
    "sample_size": str(sample_df.shape[0]),
    "sample_seed": "42",
}

# RNN
log_model_with_mlflow(
    model=rnn_model_glove,
    X_train=X_train_pad,
    y_train=y_train,
    X_test=X_test_pad,
    y_test=y_test,
    tags=tags,
    model_name="SimpleRNN_glove_embdding",
    model_version="1",
)

# LSTM
log_model_with_mlflow(
    model=lstm_model_glove,
    X_train=X_train_pad,
    y_train=y_train,
    X_test=X_test_pad,
    y_test=y_test,
    tags=tags,
    model_name="LSTM_glove_embdding",
    model_version="1",
)

# LSTM-BiDir
log_model_with_mlflow(
    model=lstm_model_bidirectional_glove,
    X_train=X_train_pad,
    y_train=y_train,
    X_test=X_test_pad,
    y_test=y_test,
    tags=tags,
    model_name="Bidirectional_LSTM_glove_embdding",
    model_version="1",
)

In [ ]:
# Pour telecharger le dossier mlruns depuis colab
# import shutil

# from google.colab import files

# # Chemin du dossier à télécharger
# dossier_a_telecharger = '/content/mlruns'

# # Nom du fichier zip
# nom_zip = 'mlruns_avance.zip'

# # Créer une archive zip
# shutil.make_archive(nom_zip.split('.')[0], 'zip', dossier_a_telecharger)

# # Télécharger le fichier zip
# files.download(nom_zip)